In [6]:
import googlemaps
from dotenv import load_dotenv
from googlemaps import convert
import os
import h3
import pandas as pd
from typing import List, Tuple

load_dotenv()
api_key = os.getenv('google_map_key')
gmaps = googlemaps.Client(key=api_key)

In [42]:
def get_sim_route(origin:list[float] , destination:list[float],color='blue'):
    route1 = gmaps.directions(origin,destination)
    route_poly = route1[0]['overview_polyline']['points']
    route_points = convert.decode_polyline(route_poly)
    route_df = pd.DataFrame(route_points, columns=['lat', 'lng'])

    def get_h3_index(row):
     return h3.geo_to_h3(row['lat'], row['lng'], 11)  
    
    route_df['h3'] = route_df.apply(get_h3_index, axis=1)
    route_df = route_df.drop_duplicates(subset=['h3'])

    l=[]
    hexes = route_df['h3'].values
    for i in range(0, len(hexes)-1):
        l.append(h3.h3_line(hexes[i], hexes[i+1]))

    flat_list = [item for sublist in l for item in sublist]
    df = pd.DataFrame(flat_list, columns=['h3'])
    for i in range(0, len(df)):
        df['lat'] = h3.h3_to_geo(df['h3'][i])[0]
        df['lng'] = h3.h3_to_geo(df['h3'][i])[1]
        df['color'] = color
    return df

In [47]:
def plot_ruta(df):
    import folium
    # Create a map centered at the first point in the dataframe
    m = folium.Map(location=[df['lat'].iloc[0], df['lng'].iloc[0]], zoom_start=15)

    # Add hexagons to the map
    for index, row in df.iterrows():
        # Get the vertices of the hexagon
        vertices = h3.h3_to_geo_boundary(row['h3'])
        
        # Convert the vertices to the format expected by folium.Polygon
        vertices = [(vertex[0], vertex[1]) for vertex in vertices]
        
        # Add the hexagon to the map
        folium.Polygon(locations=vertices, color=row['color'], fill=True, fill_color=row['color'],tooltip=row['h3']).add_to(m)
    return m

In [50]:
df1 = get_sim_route(origin=[14.57341656112588, -90.52457302750229],destination=[14.60804486286022, -90.48687551400758])
df2 = get_sim_route(origin=[14.60804486286022, -90.48687551400758],destination=[14.64242248883225, -90.51330643298236],color='red')
df = pd.concat([df1,df2])

In [52]:
plot_ruta(df)